In [1]:
import csv

csv_file_path = "SystemReq_EPS.csv"

In [2]:
def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements

In [3]:
requirements = load_csv(csv_file_path)

In [4]:
requirements

[{'id': '245100',
  'Primary Text': 'General Requirements',
  'Artifact Type': 'Heading',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': 'TRUE'},
 {'id': '245101',
  'Primary Text': "The Steering ECU is responsible for controlling and managing the vehicle's steering system. Its primary operations include receiving input from the driver (via the steering wheel), processing sensor data, and actuating steering mechanisms to ensure accurate and safe vehicle handling.\nThe ECU supports features like power-assisted steering, advanced driver assistance systems (ADAS), and emergency steering maneuvers.",
  'Artifact Type': 'SYS Information',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': ''},
 {'id': '245102',
  'Primary Text': 'Input / Output Parameters',
  'Artifact Type': 'Heading',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': 'TRUE'},
 {'id': '245103',
  'Primary Text': 'Steering Wheel Angle:  \nInput Range - ±450° (effective within ±180°)\nOu

In [5]:
from langchain_core.documents import Document

documents = [(Document(page_content=str( {
            "Primary Text": item['Primary Text'],
            "Verification_Criteria": item['Verification_Criteria'],
        }), metadata={"Requirement ID": item["id"]})) for item in requirements]

In [7]:
# from langchain_huggingface import HuggingFaceEmbeddings

# embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [7]:
import uuid
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# Add documents
client.add_documents(documents=documents, ids = ids)

['ba765540-61b2-4078-93dc-047af8bc9c93',
 '8eca4c93-a364-434d-be4e-eaf30e0a6128',
 '0955b6a3-7d94-4ada-9310-b7e89f29d79a',
 'b36dce7f-ad92-4eaa-bcea-b4d6641837a9',
 '4d3d7a19-7d8e-4e1b-9855-b29e786e1ac0',
 '206bc06b-721a-41bf-9d2f-f903ce2dcf63',
 '8b627c89-cd3d-432e-954a-dfd0e668f717',
 '302d2780-b679-477d-94ee-877eb22b6a76',
 '4b628bf6-e56d-46ec-95c2-798c9edfa606',
 '0ed5dfee-4ea0-4a3c-9031-976580b339ca',
 '458190e1-fd6d-407b-ab12-2264c04389db',
 '550e111e-e1f4-4757-82e7-af09d54dad82',
 '710b0d2c-c877-4f4a-bb1a-71785acc4a4c',
 '41aacaea-924c-4f35-8416-f35bc6488b02',
 '2d08d9d3-c5fd-4ebe-8387-17c797502a9c',
 'ba3e2d63-d06a-412d-9c3f-23d8ccecf587',
 '66dca44d-212b-4ba0-9d26-33ae4de8eaef',
 'dbabd760-eb14-4ebe-8948-a2f1cdab5c40',
 '258c1486-6d9b-4da6-8837-33e77655947d']

In [31]:
# resp = client.get(include=["embeddings"])
# print(resp)

In [13]:
query = "At what voltage Steering ECU operates ?"

In [34]:
retriever = client.as_retriever()

docs = retriever.invoke(query)
type(docs)

list

In [57]:
docs

[Document(metadata={'Requirement ID': '245101'}, page_content='{\'Primary Text\': "The Steering ECU is responsible for controlling and managing the vehicle\'s steering system. Its primary operations include receiving input from the driver (via the steering wheel), processing sensor data, and actuating steering mechanisms to ensure accurate and safe vehicle handling.\\nThe ECU supports features like power-assisted steering, advanced driver assistance systems (ADAS), and emergency steering maneuvers.", \'Verification_Criteria\': \'\'}'),
 Document(metadata={'Requirement ID': '245113'}, page_content='{\'Primary Text\': "The Steering ECU shall ensure reliable operation within the specified voltage range of the vehicle electrical system while maintaining efficient power consumption under varying load conditions.\\nOperating Range:\\n• The Steering ECU shall operate on a 12V or 48V vehicle electrical system, with the following considerations:\\n    \'- Nominal Voltage: 12V system.\\n    \'- 

In [4]:
docs[1].page_content

NameError: name 'docs' is not defined

In [15]:
def retrieve_documents(query_text, collection, top_k=4,):
    documents=[]
    retrieve_doc = collection.similarity_search(query_text, k=top_k)

    for item in retrieve_doc:
         documents.append({
                "content": item.page_content,
                "source": item.metadata,
            })  
    
    return documents

In [16]:
result = retrieve_documents("At what voltage does steering ECU operates ?",client)

In [8]:
import re

def clean_text(result):
    content_list = []
    for item in result:
        # Clean the 'content' field
        content = item['content']
        content = content.replace("'", '"')
        content = re.sub(r'[\'\-•]""', "", content)  # Remove specified characters
        content = re.sub(r'\s{2,}', ' ', content).strip()  # Normalize whitespace and trim
        item['content'] = content

        # Replace data in 'Verification_Criteria' field if it exists
        #for var in item:
        # criteria = item['Verification_Criteria']  # Update cleaned criteria in the item
        
        # combined_content = f"Primary Text: {content}\nVerification Criteria: {criteria}\n\n"
        content_list.append(content)
        final_content = ''.join(content_list)
    return final_content

In [19]:
res = clean_text(result)

In [20]:
type(res)

str

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
    )

In [10]:
from langchain.prompts import PromptTemplate

def prompt_def(task, context):
    template = """ Answer the question {task} by understanding the {context} content"""
    prompt = PromptTemplate.from_template(template=template, input_varible=["task", "context"])

    return prompt

In [11]:
def llm_call(requirement):
    #retrieving the documents
    documents = retrieve_documents(requirement,client)
    context = clean_text(documents)

    prompt = prompt_def(requirement, context)
    
    chain = prompt|llm
    output = chain.invoke(input={"task": requirement, "context": context})

    return output
    

In [17]:
llm_response = llm_call(query)

# Output the LLM's response
print("LLM Response: ", llm_response.content)

LLM Response:  The Steering ECU operates at a nominal voltage of 12V, with an operating voltage range of 10V to 24V. It is also compatible with 48V vehicle electrical systems.


In [1]:
Orches_prompt = """ You are provided with requirements of Automotive Electronic Control Unit(ECU). 
Analyze the requirements and understand the context of the it based on documents provided.
        requirements: {requirement}
        documents: {context}

Return your response in this format:

<analysis>
Explain your understanding of the requirements in detail. Focus on each point and break down it in detail.
</analysis>

<tasks>
    <task>
    <type>Primary text</type>
    <description>Write a precise description of requirement separated by numerical points</description>
    </task>
    <task>
    <type>verification</type>
    <description>Give description about how can be the respective requirement is validated</description>
    </task>
</tasks>
"""